## Initialize

### Importing Libraries

In [0]:
import gzip
import math
import numpy as np
import pickle
import tensorflow as tf

from datetime import datetime
from time import time
from tqdm import tqdm

### Defining Constants

In [0]:
base_path = '.'

### Mounting File System

In [0]:
from google.colab import drive
drive.mount('/content/drive')

base_path = './drive/My Drive/CoE202TermProject'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Loading Input Vectors

In [0]:
with gzip.open(base_path + '/datasets/train.chunk.pickle', 'rb') as f:
    train_data = pickle.load(f)

with gzip.open(base_path + '/datasets/train.image.feats.pickle', 'rb') as f:
    train_images = pickle.load(f)

train_features = train_data['features'][:]
train_labels = train_data['labels'][:]


with gzip.open(base_path + '/datasets/valid.chunk.pickle', 'rb') as f:
    valid_data = pickle.load(f)

with gzip.open(base_path + '/datasets/valid.image.feats.pickle', 'rb') as f:
    valid_images = pickle.load(f)

valid_features = valid_data['features'][:]
valid_labels = valid_data['labels'][:]

## Defining Model

### Defining Hyperparameters

In [0]:
batch_size = 512
lr = 1e-4
n_epochs = 3
embd_size = 1024
drop_prob = 0.3

### Defining Constants

In [0]:
voca_size  = 100001
valid_freq = 100
valid_freq_epoch = 1
seqlen_model = len(train_features[0][2])
seqlen_product = len(train_features[0][5])
seqlen_image = len(train_images[0])
n_classes  = np.max(train_labels) + 1

### Placeholders

In [0]:
input_brands = tf.placeholder(
    dtype=tf.float32, shape=(None, ), name='brands'
)

input_makers = tf.placeholder(
    dtype=tf.float32, shape=(None, ), name='makers'
)

input_products = tf.placeholder(
    dtype=tf.float32, shape=(None, seqlen_product), name='input_products'
)

input_product_counts = tf.placeholder(
    dtype=tf.float32, shape=(None, seqlen_product), name='input_product_counts'
)

input_images = tf.placeholder(
    dtype=tf.float32, shape=(None, 2048), name='input_images'
)

labels = tf.placeholder(
    dtype=tf.int32, shape=(None, ), name='labels'
)

is_train = tf.placeholder(
    dtype=tf.bool, name='is_train'
)

### Deep neural network

In [0]:
from tensorflow.keras.layers import Activation, BatchNormalization, \
                                    Concatenate, Dense, Dot, Dropout, \
                                    Embedding, Flatten, Reshape

# Defining Embedding Layer
embd = Embedding(voca_size, embd_size, name='embd')

# Processing Brands
x_brands = Reshape((1, ))(input_brands)
x_brands = embd(x_brands)
x_brands = BatchNormalization()(x_brands, training=is_train)
x_brands = Flatten()(x_brands)

# Processing Makers
x_makers = Reshape((1, ))(input_makers)
x_makers = embd(x_makers)
x_makers = BatchNormalization()(x_makers, training=is_train)
x_makers = Flatten()(x_makers)

# Processing Products
x_products = embd(input_products)
x_product_counts = Reshape((seqlen_product, 1))(input_product_counts)
x_products = Dot(1)([x_products, x_product_counts])
x_products = Reshape((1, embd_size))(x_products)
x_products = BatchNormalization()(x_products, training=is_train)
x_products = Flatten()(x_products)

texts = Concatenate(axis=1)([x_products, x_brands, x_makers])
text_hidden = Dense(n_classes)(texts)
text_hidden = BatchNormalization()(text_hidden, training=is_train)
text_hidden = Dropout(drop_prob)(text_hidden, training=is_train)
text_hidden = Activation('relu')(text_hidden)

# Processing Images
image_hidden = Dense(512)(input_images)
image_hidden = BatchNormalization()(image_hidden, training=is_train)
image_hidden = Dropout(drop_prob)(image_hidden, training=is_train)
image_hidden = Activation('relu')(image_hidden)

# Concatenating Images and Products
x_concat = Concatenate(axis=1)([text_hidden, image_hidden])

# Hidden Layer
hidden1 = Dense(n_classes + n_classes // 3)(x_concat)
hidden1 = BatchNormalization()(hidden1, training=is_train)
hidden1 = Activation('relu')(hidden1)

# Hidden Layer 2
hidden2 = Dense(n_classes + n_classes // 3)(hidden1)
hidden2 = BatchNormalization()(hidden2, training=is_train)
hidden2 = Dropout(drop_prob)(hidden2, training=is_train)
hidden2 = Activation('relu')(hidden2)

# Output
logits = Dense(n_classes)(hidden2)

# Softmax
preds = Activation('softmax', name='predictions')(logits)

### Loss funciton & Optimizer

In [0]:
# Softmax cross entropy loss
loss = tf.losses.softmax_cross_entropy(onehot_labels=tf.one_hot(labels, n_classes), logits=logits)

# Weight decay
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([loss] + reg_losses, name='total_loss')

# Optimizer
optm = tf.train.AdamOptimizer(lr)
train_op = optm.minimize(loss, global_step=tf.train.get_global_step(), name='step_update')
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
train_op = tf.group([train_op, update_ops])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Accuracy

In [0]:
top1_acc = tf.keras.metrics.top_k_categorical_accuracy(y_true=tf.one_hot(labels, n_classes),
                                                        y_pred=preds, k=1)
top1_acc = tf.identity(top1_acc, name='top1_acc')

top5_acc = tf.keras.metrics.top_k_categorical_accuracy(y_true=tf.one_hot(labels, n_classes),
                                                        y_pred=preds, k=5)
top5_acc = tf.identity(top5_acc, name='top5_acc')

### Batch generator

In [0]:
def generator(mode='training', use_percentage = 1):
    if mode == 'training':
        features = train_features
        labels = train_labels
        images = train_images

    elif mode == 'valid':
        features = valid_features
        labels = valid_labels
        images = valid_images
    
    elif mode == 'test':
        features = test_features
        labels = None
        images = test_images

    n_data = math.floor(len(features) * use_percentage)

    indices = np.arange(len(features))
    np.random.shuffle(indices)
    indices = indices[:n_data]
    
    for start_idx in range(0, n_data, batch_size):
        if start_idx + batch_size <= n_data:
            excerpt = indices[start_idx: start_idx + batch_size]

            if labels is not None:
                yield list(zip(*features[excerpt, :])), \
                    images[excerpt, :], \
                    labels[excerpt]
            
            else:
                yield list(zip(*features[excerpt, :])), \
                    images[excerpt, :]

def get_feed_dict(b_features, b_images, b_label, b_is_train):
    return {
        input_brands: b_features[0],
        input_makers: b_features[3],
        input_products: b_features[4],
        input_product_counts: b_features[5],
        input_images: b_images,
        labels: b_label,
        is_train: b_is_train
    }

### Training session

In [0]:
with tf.Session() as sess:
    tic = time()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(
        base_path + '/logs/{}'.format(
            datetime.now().strftime("%Y%m%d-%H%M%S")
        ),
        sess.graph
    )
    
    iteration = 0
    for epoch in range(n_epochs):
        print(
            "\n\nEpoch {0:03d} / {1:03d}\n"
            .format(epoch, n_epochs)
        )

        training_loss = []
        batch_iteration = 0
        for b_features, b_images, b_label in tqdm(
            generator(mode = 'training'),
            total = math.ceil(len(train_labels) / batch_size)
        ):
            feed_dict = get_feed_dict(b_features, b_images, b_label, True)
            
            _, train_loss = sess.run(
                [train_op, loss],
                feed_dict=feed_dict
            )

            training_loss.append(train_loss)

            iteration += 1
            batch_iteration += 1
            if batch_iteration % valid_freq != 0:
                continue

            # Write validation accuracy
            top_1_i, top_5_i, loss_i = [], [], []
            for b_features, b_images, b_label in generator('valid', 0.1):
                feed_dict = get_feed_dict(b_features, b_images, b_label, False)
                t1_acc_ij, t5_acc_ij, loss_ij = sess.run(
                    [top1_acc, top5_acc, loss],
                    feed_dict = feed_dict
                )

                top_1_i.append(t1_acc_ij)
                top_5_i.append(t5_acc_ij)
                loss_i.append(loss_ij)
            
            summary = tf.Summary()
            summary.value.add(tag="val_t1_acc", simple_value=np.mean(top_1_i))
            summary.value.add(tag="val_t5_acc", simple_value=np.mean(top_5_i))
            summary.value.add(tag="train_loss", simple_value=np.mean(
                training_loss[batch_iteration - valid_freq:]
            ))
            summary.value.add(tag="val_loss", simple_value=np.mean(loss_i))
            
            writer.add_summary(summary, iteration)
            writer.flush()

        toc = time()
        print(
            "[*] TRAIN Loss {0:.4f} | Time {1:.2f}s"
            .format(np.mean(training_loss), toc - tic)
        )
        
        if (epoch + 1) % valid_freq_epoch == 0:
            top_1, top_5 = [], []

            for b_features, b_images, b_label in generator('valid'):
                feed_dict = get_feed_dict(b_features, b_images, b_label, False)
                
                t1_acc, t5_acc = sess.run(
                    [top1_acc, top5_acc],
                    feed_dict=feed_dict
                )

                top_1.append(t1_acc)
                top_5.append(t5_acc)

            print(
                "[*] VALIDATION Top-1 Acc: {0:.4f} | Top-5 Acc: {1:.4f}"
                .format(np.mean(top_1), np.mean(top_5))
            )

    saver.save(sess, base_path + '/models/models')

  0%|          | 0/1563 [00:00<?, ?it/s]



Epoch 000 / 003



100%|█████████▉| 1562/1563 [01:51<00:00, 13.95it/s]


[*] TRAIN Loss 2.4103 | Time 116.17s


  0%|          | 2/1563 [00:00<01:55, 13.48it/s]

...

[*] TRAIN Loss 0.8254 | Time 232.27s


  0%|          | 2/1563 [00:00<01:53, 13.79it/s]

[*] VALIDATION Top-1 Acc: 0.3196 | Top-5 Acc: 0.6294


Epoch 002 / 003



  6%|▌         | 94/1563 [00:05<01:34, 15.57it/s]


KeyboardInterrupt: ignored

### Show Tensorboard

In [0]:
%load_ext tensorboard
%tensorboard --logdir "./drive/My Drive/CoE202TermProject/logs"

## Test process
### Load the test data

In [0]:
# test data
with gzip.open('./drive/My Drive/CoE202TermProject/datasets/test.chunk.pickle', 'rb') as f:
    test_data = pickle.load(f)

with gzip.open('./drive/My Drive/CoE202TermProject/datasets/valid.image.feats.pickle', 'rb') as f:
    test_images = pickle.load(f)

test_features  = test_data['features'][:]
pids = test_data['pids'][:]

### Batch generator for test dataset

In [0]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('./drive/My Drive/CoE202TermProject/models/models.meta')
    saver.restore(sess, tf.train.latest_checkpoint('./drive/My Drive/CoE202TermProject/models/'))
    DNN = tf.get_default_graph()

    preds = []
    for b_features, b_images in generator(mode='test'):
        feed_dict = {
            DNN.get_tensor_by_name('input_brands:0'): b_features[0],
            DNN.get_tensor_by_name('input_makers:0'): b_features[3],
            DNN.get_tensor_by_name('input_products:0'): b_features[4],
            DNN.get_tensor_by_name('input_product_counts:0'): b_features[5],
            DNN.get_tensor_by_name('input_prices:0'): b_features[6],
            
            DNN.get_tensor_by_name('input_images:0'): b_images,
            DNN.get_tensor_by_name('is_train:0'): False
        }

        pred = sess.run(DNN.get_tensor_by_name('predictions:0'), feed_dict=feed_dict)
        preds.extend(pred)


### Save the submission files

In [0]:
# Indexing of predictions
argpreds = np.argmax(preds, axis=1)

# Load label dictionary
with open('./drive/My Drive/CoE202TermProject/datasets/y_vocab.pickle', 'rb') as f:
    y_dict = pickle.load(f)
# y_dict = pickle.loads(open('./drive/My Drive/CoE202TermProject/datasets/y_vocab.pickle').read())

# Inverse label dictionary
inv_y_dict = dict((y,x) for x,y in y_dict.items())
submissions = [inv_y_dict[argpred] for argpred in argpreds]

# Write the results to 'submissions.csv'
f = open('./drive/My Drive/CoE202TermProject/submissions.csv', 'w')
for i, j in zip(pids, submissions):
    line = '{},{}\n'.format(i,j)
    f.write(line)
f.close()

You should submit the 'submissions.csv' file and 4 tf.save files ('checkpoint', 'dnn_models.data', 'dnn_models.index', 'dnn_models.meta') in models folder